In [1]:
# load environment
import Pkg; Pkg.activate(".")

  Activating project at `~/Documents/PhD/MyPublications/2025/SISSI_II_Local_Bubble_Age/SISSI-II-Local-Bubble`


In [2]:
Pkg.instantiate();

Precompiling project...
  ✓ LaTeXStrings
  ✓ PtrArrays
  ✓ DefineSingletons
  ✓ InverseFunctions
  ✓ CompositionsBase
  ✓ ArgCheck
  ✓ Compat
  ✓ Requires
  ✓ ConstructionBase
  ✓ JLLWrappers
  ✓ OrderedCollections
  ✓ InitialValues
  ✓ AliasTables
  ✓ ConstructionBase → ConstructionBaseLinearAlgebraExt
  ✓ Compat → CompatLinearAlgebraExt
  ✓ Adapt
  ✓ IrrationalConstants
  ✓ Conda
  ✓ CFITSIO_jll
  ✓ Baselet
  ✓ Tables
  ✓ CompositionsBase → CompositionsBaseInverseFunctionsExt
  ✓ InverseFunctions → InverseFunctionsTestExt
  ✓ Adapt → AdaptSparseArraysExt
  ✓ ColorTypes
  ✓ LogExpFunctions
  ✓ CFITSIO
  ✓ InverseFunctions → InverseFunctionsDatesExt
  ✓ Referenceables
  ✓ LogExpFunctions → LogExpFunctionsInverseFunctionsExt
  ✓ MacroTools
  ✓ DataStructures
  ✓ FITSIO
  ✓ SortingAlgorithms
  ✓ Setfield
  ✓ Healpix
  ✓ ColorVectorSpace
  ✓ SplittablesBase
  ✓ Accessors
  ✓ StatsBase
  ✓ Accessors → LinearAlgebraExt
  ✓ Colors
  ✓ Accessors → TestExt
  ✓ BangBang
  ✓ BangBang → BangBangT

# Packages & Includes

In [ ]:
# Julia packages
using PyPlot, ColorSchemes, Colors
using DelimitedFiles
using Random
using LinearAlgebra
using StatsBase
using ThreadsX
using FITSIO

# Python packages
using PyCall
gpy = pyimport("galpy")
pot = pyimport("galpy.potential")
gpy_coords = pyimport("galpy.util.coords")
orbit = pyimport("galpy.orbit")

PyObject <module 'labellines' from '/home/leonard/miniconda3/lib/python3.9/site-packages/labellines/__init__.py'>

# Constants

## Numerical Constants

In [ ]:
const TINY = nextfloat(0.0)

5.0e-324

## Physical Constants

In [4]:
# Some physical constants (in cgs units)
const mH   = 1.67262192e-24        # proton/hydrogen mass
const X_H  = 0.76                  # primordial hydrodgen fraction
const Y_He = (1 - X_H) / X_H / 4   # primordial abundance of helium
const μ    = (1 / X_H) * mH        # primordial mean atomic weight
const kB   = 1.380649e-16          # Boltzmann constant

1.380649e-16

## Units

In [5]:
# All units are in cgs unless otherwise specified
# Length
const μm  = 1e-4
const pc  = 3.086e18
const kpc = 1e3 * pc

# Time
const yr  = 3.155e7
const kyr = 1e3 * yr
const Myr = 1e6 * yr

# Mass
const Msol = 1.989e33

# Velocity
const km_s    = 1e5

# Density
const Dunit = 1653.19183 # Conversion factor between differential extinction & Hydrogen number density (O'Neill et al. 2024)

1653.19183

# Function Definitions

## Probability Density Functions

In [ ]:
# smooth quantile-function & pdf
quant(y::Real; β::Real=0.7, γ::Real=1, a::Vector{<:Real}=[1, 1, 1]) = y^β / (1-y)^γ * dot(a, [1, y, sin(π * y)])
prob(y::Real; β::Real=0.7, γ::Real=1, a::Vector{<:Real}=[1, 1, 1])  = ((β + (γ-β) * y) * y^(β-1) / (1-y)^(γ+1) * dot(a,[1, y, sin(π * y)]) + y^β / (1-y)^γ * dot(a, [0, 1, π * cos(π * y)]))^-1

In [ ]:
# Marginal PDF for discrete events (SNe)
# Given the p0 = P(t < t_delay)
# To get the pdf, given t > t_delay, I need to simply multiply the value for N > 0 with (1 - p0) and at N=0 P(N=0|t > t_delay) = (P_0 - p0) / (1 - p0)
# similarly for the quantile function I can just evaluate at p -> p0 + (1-p0) * p.

"Return the probability of exactly N supernova given the percentile vector Q and the probability of getting exactly 0 SNe"
function PDF_N_SN(N; Q::Vector{<:Real}, P_0::Real)
    if N == 0
        return P_0
    else
        if N <= Q[1]
            return (0.16-P_0) / Q[1]
        elseif N <= Q[2]
            p = Q[1] == 0 ? (0.34-P_0) : 0.34
            return p / (Q[2] - Q[1]) 
        elseif N <= Q[3]
            p = Q[2] == 0 ? (0.34-P_0) : 0.34
            return p / (Q[3] - Q[2])
        else
            norm = Q[3] == 0.0 ? (1-P_0) : 0.16
            return norm * (ℯ-1) * exp(-(N-Q[3]))
        end
    end
end

"Return the Quantile of the SN pdf."
function Quantile_N_SN(p::Real; Q::Vector{<:Real}, P_0::Real)
    if p <= P_0
        return 0
    elseif p <= 0.16
        return ceil(Int, (p - P_0) / (0.16 - P_0) * Q[1])
    elseif p <= 0.5
        p0 = max(P_0, 0.16)
        return Q[1] + ceil(Int, (p - p0) / (0.5 - p0) * (Q[2] - Q[1]))
    elseif p <= 0.84
        p0 = max(P_0, 0.5)
        return Q[2] + ceil(Int, (p - p0) / (0.84 - p0) * (Q[3] - Q[2]))
    else
        p0 = max(P_0, 0.84)
        return Q[3] + ceil(Int, log((1 - p0) / (1-p)))
    end
end

In [ ]:
"Return the probability of P(t_age < t_delay) and the probability of getting exactly 0 SNe."
function compute_P0(N_SN::Vector{<:Real}, P_age::Tuple{<:Real, <:Real, <:Vector{<:Real}}, age::Vector{<:Real}; 
                    p::Vector{<:Real}=[0.16, 0.5, 0.84], t_delay::Real=3.0, TOL::Real=1e-4)
    # lower limit of P0 based on zero quantiles
    P_0 = maximum([N_SN[i] == 0 ? p[i] : 0.0 for i in eachindex(p)])

    # lower limit of P0 based on non-zero probabiliy of N=0
    i1 = findfirst(N_SN .> 0)
    if !isnothing(i1)
        dp = i1 == 1 ? p[1] : p[i1] - p[i1-1]
        P_0 = max(P_0, dp / (1+N_SN[i1]))
    end

    # get probability of t_age < t_delay    
    # get quantile function
    q(y) = quant(y, β=P_age[1], γ=P_age[2], a=P_age[3])

    # get initial guess
    i0 = findfirst(age .> t_delay)
    p0 = !isnothing(i0) ? (i0 > 1 ? p[i0-1] : 0.0) : p[end]
    p1 = !isnothing(i0) ? p[i0] : 1.0
    pn = 0.5 * (p0 + p1)

    while p1 - p0 > TOL
        pn = 0.5 * (p0 + p1)

        if q(pn) < t_delay
            p0 = pn
        else
            p1 = pn
        end 
    end

    return pn, max(pn, P_0)
end

In [ ]:
"Return the probability of P(t_age < t)."
function P_exists(P_age::Tuple{<:Real, <:Real, <:Vector{<:Real}}, t::Real, age::Vector{<:Real}; 
                    p::Vector{<:Real}=[0.16, 0.5, 0.84], TOL::Real=1e-4)
    # get probability of t_age < t_delay    
    # get quantile function
    q(y) = quant(y, β=P_age[1], γ=P_age[2], a=P_age[3])

    # get initial guess
    i0 = findfirst(age .> t)
    p0 = !isnothing(i0) ? (i0 > 1 ? p[i0-1] : 0.0) : p[end]
    p1 = !isnothing(i0) ? p[i0] : 1.0
    pn = 0.5 * (p0 + p1)

    while p1 - p0 > TOL
        pn = 0.5 * (p0 + p1)

        if q(pn) < t
            p0 = pn
        else
            p1 = pn
        end 
    end

    return pn
end

## I/O

In [ ]:
"Read results data from csv file and format into data that one can work with."
function read_data(fpath::String)
    # read out full results table
    data = readdlm(fpath, ',', skipstart=1)[1:end-18, :]

    # assigned family
    family = data[:, 66]
    age = 10.0.^(data[:, 48:50]) / 1e6
    positions = Float64.(data[:, 69:71])
    velocity  = Float64.(data[:, 72:74])
    mass      = Float64.(data[:, 82:84])
    N_SN      = Float64.(data[:, 85:87])

    # get SN Rate
    R_SN = zeros(size(N_SN))
    for i in eachindex(family)
        R_SN[i, 1] = N_SN[i, 1] / min(40.0, age[i, 3])
        R_SN[i, 2] = N_SN[i, 2] / min(40.0, age[i, 2]) 
        R_SN[i, 3] = N_SN[i, 3] / min(40.0, age[i, 1])
    end
    
    # Split up by clusters
    mask_Cr135 = family .== "cr135"
    Cr135 = Dict("age"    => age[mask_Cr135, :],
                 "mass"   => mass[mask_Cr135, :],
                 "N_SN"   => N_SN[mask_Cr135, :],
                 "R_SN"   => R_SN[mask_Cr135, :],
                 "coords" => positions[mask_Cr135, :],
                 "vel"    => velocity[mask_Cr135, :])  

    mask_M6 = family .== "m6"
    M6 = Dict("age"    => age[mask_M6, :],
              "mass"   => mass[mask_M6, :],
              "N_SN"   => N_SN[mask_M6, :],
              "R_SN"   => R_SN[mask_M6, :],
              "coords" => positions[mask_M6, :],
              "vel"    => velocity[mask_M6, :])  
        
    mask_αPer = family .== "alphaPer"
    αPer = Dict("age"    => age[mask_αPer, :],
                "mass"   => mass[mask_αPer, :],
                "N_SN"   => N_SN[mask_αPer, :],
                "R_SN"   => R_SN[mask_αPer, :],
                "coords" => positions[mask_αPer, :],
                "vel"    => velocity[mask_αPer, :])


    return Cr135, M6, αPer
end

read_mean_dust_map_file

## Utility

In [ ]:
"Return N logarithmically spaced points between logx0 and logx1."
LogRange(logx0::Real, logx1::Real, N::Integer) = 10.0.^(LinRange(logx0, logx1, N))

"convert healpix pixel to angle"
get_angle(ihp::Integer; nside::Integer=Nside) = pix2ang(Map{Float64, NestedOrder}(nside), ihp)

"get coordinates in heliocentric Galactic cartesian coordinates"
get_coordinates_3d(θ::Real, ϕ::Real) = [sin(θ) * cos(ϕ), sin(θ) * sin(ϕ), cos(θ)]

"Check if a number is NaN or Inf."
isbad(x::Real) = isnan(x) || isinf(x)

"Check if an array has NaN or Inf values."
hasbad(a::Array{<:Number}) = sum(isbad.(a)) .> 0

"Return linear interpolation operator from domain to target space"
function linear_interpolator(x_target::Vector{<:Real}, x_domain::Vector{<:Real})
    L = zeros(length(x_target), length(x_domain))
    
    for j in eachindex(x_target)
        for i in 1:length(x_domain)-1
            if x_domain[i] <= x_target[j] <= x_domain[i+1]
                Δx = x_domain[i+1] - x_domain[i]
                L[j, i]   = (x_domain[i+1] - x_target[j]) / Δx
                L[j, i+1] = (x_target[j]   - x_domain[i]) / Δx
            end
        end
    end
    
    return L
end

"Return a color given a colorscheme and a value"
function get_color(colorscheme, value; norm=nothing)
    norm = isnothing(norm) ? f(x) = x : norm 
    color = colorscheme[norm(value)]
    return (red(color), green(color), blue(color))
end

hasbad

## Orbit Integration

In [ ]:
"Return the orbit of a particle starting of at heliocentric position xyz_hel [pc] with heliocentric speed UVW [km/s] as a function of time ts [Myr]."
function get_orbit(XYZ_hel::Vector{<:Real}, UVW::Vector{<:Real}, T::Vector{<:Real}; 
                   R_sun::Real=8.122, z_sun::Real=0.0208, Vc::Real=236.0, v_sun::Vector{<:Real}=[-11.1, 12.4, 7.25])
    # get orbital timescale in Myr
    t_unit = (R_sun * kpc) / (Vc * km_s) / Myr
    
    # convert to natural units
    xyz_hel = XYZ_hel / (1e3 * R_sun)
    uvw  = UVW / Vc
    zsun = z_sun / R_sun
    vsun = v_sun / Vc + [0, 1, 0]
    t    = T / t_unit
    
    
    # get coordinates in galactocentric Cartesian coordinates
    x, y, z = gpy_coords.XYZ_to_galcenrect(xyz_hel..., Xsun=1.0, Zsun=zsun)

    # get coordinates in galactocentric cylindrical coordinates
    R, ϕ, z = gpy_coords.XYZ_to_galcencyl(xyz_hel..., Xsun=1.0, Zsun=zsun)

    # get galactocentric cylindrical speeds
    vR, vT, vz = gpy_coords.vxvyvz_to_galcencyl(uvw..., x, y, z, vsun=vsun, Xsun=1.0, Zsun=zsun)

    # get 6D coordinate vector
    c_6D = [R, vR, vT, z, vz, ϕ]

    # generate orbit object and integrate orbit
    o = orbit.Orbit(c_6D, ro=R_sun, zo=z_sun, vo=Vc, solarmotion=v_sun)
    o.integrate(t, pot.MWPotential2014, method="odeint")
    
    return o
end

"Return positions in heliocentric frame given 6D coordinates in natural units"
function get_XYZ_hel(c_6D::Vector{<:Real}; R_sun::Real=8.122, z_sun::Real=0.0208, Vc::Real=236.0, v_sun::Vector{<:Real}=[-11.1, 12.4, 7.25])
    return [gpy_coords.galcencyl_to_XYZ(c_6D[1], c_6D[6], c_6D[4], Xsun=1, Zsun=z_sun / R_sun)...] * 1e3 * R_sun
end

"Return position of circular orbit centered around sun at t=0 as a function of time."
function cicular_orbit(t::Real; R_sun::Real=8.122, z_sun::Real=0.0208, Vc::Real=236.0)
    t_unit = (R_sun * kpc) / (Vc * km_s) / Myr
    τ      = t / t_unit
    return [R_sun * (1-cos(τ)), R_sun * sin(τ), z_sun] * 1e3
end

# Loading the data

In [ ]:
# Enter here the path to the data directory, where the cluster data files (cluster_sample_data.csv) are located
path_data = "path/to/data/"

# Names of the different cluster families
families = ["Cr135", "M6", "αPer"]

"path/to/data"

## Star Clusters

In [ ]:
# Load star cluster data
Cr135, M6, αPer = read_data(path_data * "cluster_sample_data.csv")
Families = Dict("Cr135" => Cr135, "M6" => M6, "αPer" => αPer)

## Local Bubble

In [ ]:
# Load Local Bubble Shell data
LB_shell = readdlm(path_data * "LB_shell.csv")
x_LB = LB_shell[:, 1]
y_LB = LB_shell[:, 2]
z_LB = LB_shell[:, 3];

### Compute 2D projection

In [ ]:
# physical extent of projection [-500 pc, 500 pc], ~2 pc resolution
xbins = LinRange(-500, 500, 513)

# compute projection
h = fit(Histogram, (x_LB, y_LB), weights(ones(size(x_LB))), (xbins, xbins)).weights

# rescale for better contrast
h /= maximum(h)
h = h.^0.4;

# projection map
LB = zeros(size(h)..., 4)
LB[:, :, 3] .= 1 # blue channel
LB[:, :, 4] = permutedims(h); # alpha channel

## Gas Distribution

In [ ]:
# Load gas distribution
f = FITS(path_data * "mean_and_std_xyz.fits")
rho_xyz = read(f[2]) * Dunit;
rho_xyz[isnan.(rho_xyz)] .= 0.0
close(f)
Σ_gas = dropdims(sum(rho_xyz, dims=(3,)) * 2 * pc^3 * μ / Msol, dims=(3,));

# Analysis

In [ ]:
# Model parameters
t_delay  = 3.0   # Minimum delay timescale [Myr]
t_active = 40.0  # Time delay of most delayed type-II SNe [Myr]

## Assigning PDFs

In [ ]:
# which probability quantiles are considered?
p = [0.16, 0.5, 0.84]

In [ ]:
# Matrix for weight assignment of mass & age distributions
M = zeros(3, 3)
for i in 1:3 M[i, :] = [1, p[i], sin(π * p[i])] end
M = inv(M)

In [ ]:
# Vector for weighting Q
γ=1
β=0.7

w = @. (1 - p)^γ / p^β

In [ ]:
# compute PDF parameters for each cluster
PDFs = Dict{String, Dict{String, Vector{Tuple{<:Real, <:Real, <:Vector{<:Real}}}}}()

for family in families
    mass  = Families[family]["mass"]
    age   = Families[family]["age"]
    N_SN  = Families[family]["N_SN"]

    # Number of clusters
    Ncl = length(mass[:, 2])

    P_mass = Vector{Tuple{<:Real, <:Real, <:Vector{<:Real}}}(undef, Ncl)
    P_age  = Vector{Tuple{<:Real, <:Real, <:Vector{<:Real}}}(undef, Ncl)
    P_SN   = Vector{Tuple{<:Real, <:Real, <:Vector{<:Real}}}(undef, Ncl)

    for i in eachindex(mass[:, 2])
        # compute parameters of mass & age PDFs
        P_mass[i]  = (β, γ, M * (w .* mass[i, :]))
        P_age[i]   = (β, γ, M * (w .* age[i, :]))
        p0, P_0    = compute_P0(N_SN[i, :], P_age[i], age[i, :], p=p, t_delay=t_delay)
        P_SN[i]    = (p0, P_0, N_SN[i, :])
    end

    # save results
    PDFs[family] = Dict{String, Vector{Tuple{<:Real, <:Real, <:Vector{<:Real}}}}()
    PDFs[family]["P_mass"] = P_mass
    PDFs[family]["P_age"]  = P_age
    PDFs[family]["P_SN"]   = P_SN
end

## Drawing Samples

In [ ]:
# Number of statistical sample points
Nsample = 10^6

samples = Dict{String, Dict{String, Matrix{<:Real}}}()

for family in families
    # Parameters for pdfs
    P_mass = PDFs[family]["P_mass"]
    P_age  = PDFs[family]["P_age"]
    P_SN   = PDFs[family]["P_SN"]

    # Number of clusters
    Ncl = length(P_mass)

    # draw random numbers
    mass_samples = rand(Ncl, Nsample)
    age_samples  = rand(Ncl, Nsample)
    N_SN_samples = zeros(Int, Ncl, Nsample)

    for i in eachindex(P_mass)
        # compute sample points
        # mass
        q_mass(y) = quant(y, β=P_mass[i][1], γ=P_mass[i][2], a=P_mass[i][3])
        mass_samples[i, :] = q_mass.(mass_samples[i, :])

        # age
        q_age(y) = quant(y, β=P_age[i][1], γ=P_age[i][2], a=P_age[i][3])
        age_samples[i, :] = q_age.(age_samples[i, :])

        yes_SNe = age_samples[i, :] .> t_delay
        
        # N_SN
        q_N_SN(y) = Quantile_N_SN(P_SN[i][1] + (1 - P_SN[i][1]) * y, Q=P_SN[i][3], P_0=P_SN[i][2])
        N_yes = sum(yes_SNe)
        N_SN_samples[i, yes_SNe] .= q_N_SN.(rand(N_yes))
    end

    # save results
    samples[family] = Dict{String, Matrix{<:Real}}()
    samples[family]["mass"] = mass_samples
    samples[family]["age"]  = age_samples
    samples[family]["N_SN"] = N_SN_samples
end

## Compute Orbits

In [ ]:
# specify orbit snapshots
t_orbit = [(0:-0.1:-20.0)...]

In [ ]:
# set SN delay timescale
orbits = Dict{String, Vector{PyObject}}()

for family in families
    XYZ  = Families[family]["coords"]
    UVW  = Families[family]["vel"]

    # Number of clusters
    Ncl = size(XYZ)[1]

    # get orbit vector
    os = Vector{PyObject}(undef, Ncl)

    for i in eachindex(os)
        # compute parameters of mass & age PDFs
        os[i] = get_orbit(XYZ[i, :], UVW[i, :], t_orbit)
    end

    # save results
    orbits[family] = os
end

### Geometrical cut: The Local Bubble

In [ ]:
# geometry cut
xyz     = Families["αPer"]["coords"]
mask_LB = xyz[:, 2] + 0.8 * xyz[:, 1] .< 320

### SN Rate along orbit

In [ ]:
# Ntime
Ntime = length(t_orbit)

# create output object
SN_Rates = Dict{String, Vector{Matrix{Float64}}}()

for family in families
    # Parameters for age pdf
    age = samples[family]["age"]
    N_SN = samples[family]["N_SN"]

    R_SN = @. N_SN / max(min(age, t_active) - t_delay, TINY)

    # Number of clusters
    Ncl = size(age)[1]

    # SFH samples
    SNR_History = Vector{Matrix{Float64}}(undef, Ncl)

    for icl in 1:Ncl
        # SFH samples
        SNR_sample = zeros(Ntime, Nsample)

        Threads.@threads for is in 1:Nsample
            it = -t_active .<= (t_orbit .- age[icl, is]) .<= -t_delay
            SNR_sample[it, is] .= R_SN[icl, is]
        end

        # compute statistics
        SNR_stat = zeros(Ntime, 3)
        Threads.@threads for it in eachindex(t_orbit)
            SNR_stat[it, :] = quantile(SNR_sample[it, :], p)
        end
                
        # store results
        SNR_History[icl] = SNR_stat
    end

    # store SN Rate History of all clusters
    SN_Rates[family] = SNR_History
end

### Active Probability along orbit

In [ ]:
# Ntime
Ntime = length(t_orbit)

# create output object
P_active = Dict{String, Vector{Vector{Float64}}}()

for family in families
    # Parameters for age pdf
    age   = Families[family]["age"]
    P_age = PDFs[family]["P_age"]

    # Number of clusters
    Ncl = length(P_age)

    # active probability
    p_active = Vector{Vector{Float64}}(undef, Ncl)

    for icl in 1:Ncl
        # Get the probability that the cluster was formed within the past 40 Myr
        p_active[icl] = [P_exists(P_age[icl], -ti+t_active, age[icl, :]) - P_exists(P_age[icl], -ti+t_delay, age[icl, :])   for ti in t]
    end

    # store active probability distribution of all clusters
    P_active[family] = p_active
end

## Star-formation History

In [ ]:
# Specify times at which SFH is to be sampled
t_SFH = [LinRange(0, 200, 1000)...]

### Cluster Families

In [ ]:
# Ntime
Ntime = length(t_SFH)

# create output object
SFH  = Dict(family => zeros(Ntime, 3) for family in families)

# cumulative probability
cp = LinRange(0, 0.999, 10 * Ntime)

for family in families
    # Parameters for age pdf
    P_age = PDFs[family]["P_age"]

    # mass samples
    mass = samples[family]["mass"]

    # SFH samples
    SFH_sample = zeros(Ntime, Nsample)

    for i in eachindex(P_age)
        # get the age distribution of this cluster
        t     = quant.(cp, β=P_age[i][1], γ=P_age[i][2], a=P_age[i][3])
        pdf_t = prob.(cp, β=P_age[i][1], γ=P_age[i][2], a=P_age[i][3])

        # get interpolatation onto time
        pdf = linear_interpolator(t_SFH, t) * pdf_t

        Threads.@threads for isample in eachindex(mass[i, :])
            SFH_sample[:, isample] += mass[i, isample] * pdf
        end
    end

    # save results
    Threads.@threads for it in eachindex(t_SFH)
        SFH[family][it, :] = quantile(SFH_sample[it, :], p)
    end
end

### Local Bubble

In [ ]:
# Ntime
Ntime = length(t_SFH)

# create output object
SFH_LB = zeros(Ntime, 3)

# cumulative probability
cp = LinRange(0, 0.999, 10 * Ntime)

# Parameters for age pdf
P_age = PDFs["αPer"]["P_age"][mask_LB]

# mass samples
mass = samples["αPer"]["mass"][mask_LB, :]

# SFH samples
SFH_sample = zeros(Ntime, Nsample)

for i in eachindex(P_age)
    # get the age distribution of this cluster
    t     = quant.(cp, β=P_age[i][1], γ=P_age[i][2], a=P_age[i][3])
    pdf_t = prob.(cp, β=P_age[i][1], γ=P_age[i][2], a=P_age[i][3])

    # get interpolatation onto time
    pdf = linear_interpolator(t_SFH, t) * pdf_t

    Threads.@threads for isample in eachindex(mass[i, :])
        SFH_sample[:, isample] += mass[i, isample] * pdf
    end
end

# save results
Threads.@threads for it in eachindex(t_SFH)
    SFH_LB[it, :] = quantile(SFH_sample[it, :], p)
end

## Supernova Rate

In [ ]:
# Specify times at which SNR is to be sampled
t_SNR = [LinRange(0, 200, 1000)...]

### Cluster Families

In [ ]:
# Ntime
Ntime = length(t_SNR)

# create output object
SNR  = Dict(family => zeros(Ntime, 3) for family in families)

for family in families
    # Parameters for age pdf
    age = samples[family]["age"]

    # mass samples
    N_SN = samples[family]["N_SN"]

    R_SN = @. N_SN / max(min(age, t_active) - t_delay, TINY)

    # SFH samples
    SNR_sample = zeros(Ntime, Nsample)

    Threads.@threads for ij in CartesianIndices(age)
        it = -t_active .<= (t_SNR .- age[ij]) .<= -t_delay
        SNR_sample[it, ij[2]] .+= R_SN[ij]
    end

    # save results
    Threads.@threads for it in eachindex(t_SNR)
        SNR[family][it, :] = quantile(SNR_sample[it, :], p)
    end
end

### Local Bubble

In [ ]:
# Ntime
Ntime = length(t_SNR)

# create output object
SNR_LB  = zeros(Ntime, 3)

# estimate SN Rate
age = samples["αPer"]["age"][mask_LB, :]
N_SN = samples["αPer"]["N_SN"][mask_LB, :]
R_SN = @. N_SN / max(min(age, t_active) - t_delay, TINY)

# SFH samples
SNR_sample = zeros(Ntime, Nsample)

Threads.@threads for ij in CartesianIndices(age)
    it = -t_active .<= (t_SNR .- age[ij]) .<= -t_delay
    SNR_sample[it, ij[2]] .+= R_SN[ij]
end

# save results
Threads.@threads for it in eachindex(t_SNR)
    SNR_LB[it, :] = quantile(SNR_sample[it, :], p)
end

In [ ]:
# Integrate to get number of SNe in time intervals
dt_SNR = t_SNR[2] - t_SNR[1]
N_SN_5  = round.(Int, [0.5 * sum(SNR_LB[1:25, i] + SNR_LB[2:26, i]) * dt_SNR for i in 1:3])
N_SN_10 = round.(Int, [0.5 * sum(SNR_LB[26:50, i] + SNR_LB[27:51, i]) * dt_SNR for i in 1:3])
N_SN_15 = round.(Int, [0.5 * sum(SNR_LB[51:75, i] + SNR_LB[52:76, i]) * dt_SNR for i in 1:3])
N_SN_20 = round.(Int, [0.5 * sum(SNR_LB[76:100, i] + SNR_LB[77:101, i]) * dt_SNR for i in 1:3]);

# Plots

In [ ]:
# Plotting Parameters
fontsize = 18
output_directory = "path/to/figures/"
colors  = Dict("Cr135" => "orange", "M6" => "royalblue", "αPer" => "crimson")
markers = Dict("Cr135" => "d", "M6" => "*", "αPer" => "o")

## C.1 Example PDF

In [ ]:
# Mass vs. SNe
fig = plt.figure(figsize=(6, 4), num=1, clear=true)

# example cluster
Q = Families["αPer"]["age"][1, :]
y = LinRange(0, 1, 10000)

# axis limits
tlim = [0, 200]
ylim = [0.0, 0.015]

ax = gca()

# plot pdf for various values of β
βs  = 0.4:0.3:1.0
cβ  = Dict(0.1 => "cyan", 0.4 => "darkblue", 0.7 => "black", 1.0 => "darkred")
lwβ = Dict(0.1 => 1, 0.4 => 1, 0.7 => 3, 1.0 => 1)

for b in βs
    wb = @. (1 - p)^γ / p^b
    a  = M * (wb .* Q)

    x = quant.(y, β=b, γ=γ, a=a)
    z = prob.(y, β=b, γ=γ, a=a)
    
    ax.plot(x, z, linestyle="-", color=cβ[b], linewidth=lwβ[b], zorder=2, label=L"\mathrm{β = %$(b), \ γ=1.0}")
end

# plot pdf for various values of γ
γs  = [0.7, 1.1]
cγ  = Dict(0.7 => "lightblue", 1.0 => "black", 1.1 => "orange")

for g in γs
    wb = @. (1 - p)^g / p^β
    a  = M * (wb .* Q)

    x = quant.(y, β=β, γ=g, a=a)
    z = prob.(y, β=β, γ=g, a=a)
    
    ax.plot(x, z, linestyle="--", color=cγ[g], linewidth=1, zorder=2, label=L"\mathrm{β = 0.7, \ γ=%$(g)}")
end

for q in Q ax.axvline(q, linewidth=2, color="blue", linestyle=":", zorder=1) end

# decorate axes
ax.set_ylabel("PDF", fontsize=fontsize)
ax.set_xlabel(L"\mathrm{Age \ [Myr]}", fontsize=fontsize)
ax.legend(loc="best", frameon=false, handlelength=1.5, fontsize=0.7fontsize)
ax.tick_params(axis="both", labelsize=fontsize)

# set axis limits
ax.set_xlim(tlim...)
ax.set_ylim(ylim...)
ax.set_yticklabels([])

savefig(output_directory * "pdf_example.pdf", bbox_inches="tight", dpi=300)
fig.clear()

## C.2 Star-Formation History & Supernova Rate

In [1]:
t_delays = [0.0, 3.0, 5.0]
c_delay  = Dict(0.0 => "lightblue", 1.0 => "purple", 3.0 => "black", 5.0 => "darkred")
lw_d     = Dict(0.0 => 1, 1.0 => 1, 3.0 => 3, 5.0 => 1)
α_d      = Dict(0.0 => 0.15, 1.0 => 0.15, 3.0 => 0.3, 5.0 => 0.15)

Dict{Float64, Float64} with 4 entries:
  0.0 => 0.15
  5.0 => 0.15
  3.0 => 0.3
  1.0 => 0.15

Note that the below plot is done for a single value of t_delay.   
For the paper we reran the above analysis multiple times for different values of t_delay and plotted them all in the same manner as below.   
If this is desired as well it is straightforward to extend the scripts accordingly.   

In [ ]:
# create figure
fig = plt.figure(figsize=(6, 12), num=1, clear=true)

# axis limits
tlim   = [-40, 0]
SFHlim = [0.0, 1.6]
SNRlim = [0.2, 20]
dtlim  = [0.05, 4]

family="αPer"

axs = fig.subplots(ncols=1, nrows=3)
subplots_adjust(wspace=0.0, hspace=0.0)

# reduce margins
margins(0,0)

# plot SFH
ax = axs[1]

ax.plot(-t_SFH, SFH[family][:, 2] / 1e3, color="black", linewidth=3, zorder=2)
ax.fill_between(-t_SFH, SFH[family][:, 1] / 1e3, SFH[family][:, 3] / 1e3, color="black", zorder=1, alpha=0.3)

# Local Bubble
ax.plot(-t_SFH, SFH_LB[:, 2] / 1e3, color="purple", linewidth=3, zorder=2)
ax.fill_between(-t_SFH, SFH_LB[:, 1] / 1e3, SFH_LB[:, 3] / 1e3, color="purple", zorder=1, alpha=0.3)

# add legend handle for previous model
ax.plot([NaN], [NaN], linestyle=":", linewidth=2, color="orange", zorder=0, label=L"\mathrm{Z+22}")
ax.fill_between([NaN], [NaN], color="royalblue", zorder=-1, alpha=0.2, label="This Work")

# decorate axes
ax.set_ylabel(L"\mathrm{SF \ Rate \ [10^{3} \ M_{\odot} / Myr]}", fontsize=fontsize)
ax.legend(loc="best", frameon=false, handlelength=1.5, fontsize=fontsize)
ax.tick_params(axis="both", labelsize=fontsize)

# set axis limits
ax.set_xlim(tlim...)
ax.set_ylim(SFHlim...)

# plot the SN-Rate
ax = axs[2]

t_d = t_delay
#for t_d in t_delays
    ax.plot(-t_SNR, SNR[family][:, 2], color=c_delay[t_d], zorder=2, linewidth=lw_d[t_d])
    ax.fill_between(-t_SNR, SNR[family][:, 1], SNR[family][:, 3], color=c_delay[t_d], zorder=1, alpha=α_d[t_d])
#end

# Local Bubble
ax.plot(-t_SNR, SNR_LB[:, 2], color="purple", zorder=2, linewidth=3, label="Local Bubble")
ax.fill_between(-t_SNR, SNR_LB[:, 1], SNR_LB[:, 3], color="purple", zorder=1, alpha=0.3)

# Zucker model
ax.axhline(1 / 1.06, linestyle=":", linewidth=2, color="orange", zorder=0)
ax.fill_between([-40, 0], [1 / (1.06 + 0.63), 1 / (1.06+0.63)], [1 / (1.06 - 0.39), 1 / (1.06-0.39)], color="orange", zorder=-1, alpha=0.2)
ax.fill_between([-40, 0], [1 / 0.1, 1 / 0.1], [1 / 0.3, 1 / 0.3], color="royalblue", zorder=-1, alpha=0.2)

# decorate axes
ax.set_ylabel(L"\mathrm{SN \ Rate \ [Myr^{-1}]}", fontsize=fontsize)
ax.legend(loc="best", frameon=false, handlelength=1.5, fontsize=fontsize)
ax.tick_params(axis="both", labelsize=fontsize)

# set axis limits
ax.set_xlim(tlim...)
ax.set_ylim(SNRlim...)
ax.set_yscale("log")

# plot the SN-time
ax = axs[3]

#for t_d in t_delays
    ax.plot(-t_SNR, 1.0 ./ SNR[family][:, 2], color=c_delay[t_d], linewidth=lw_d[t_d], zorder=2, label=" $t_d")
    ax.fill_between(-t_SNR, 1.0 ./ SNR[family][:, 1], 1.0 ./ SNR[family][:, 3], color=c_delay[t_d], zorder=1, alpha=α_d[t_d])
#end

# Local Bubble
ax.plot(-t_SNR, 1.0 ./ SNR_LB[:, 2], color="purple", zorder=2, linewidth=3)
ax.fill_between(-t_SNR, 1.0 ./ SNR_LB[:, 1], 1.0 ./ SNR_LB[:, 3], color="purple", zorder=1, alpha=0.3)

# Zucker model
ax.axhline(1.06, linestyle=":", linewidth=2, color="orange", zorder=0)
ax.fill_between([-40, 0], [(1.06 + 0.63), (1.06+0.63)], [(1.06 - 0.39), (1.06-0.39)], color="orange", zorder=-1, alpha=0.2)
ax.fill_between([-40, 0], [0.1, 0.1], [0.3, 0.3], color="royalblue", zorder=-1, alpha=0.2)

# decorate axes
ax.set_ylabel(L"\mathrm{\Delta t_{SN} \ [Myr]}", fontsize=fontsize)
ax.set_xlabel(L"\mathrm{Time \ [Myr]}", fontsize=fontsize)
ax.legend(loc="best", frameon=false, handlelength=1.5, fontsize=fontsize, title=L"\mathrm{t_{delay} \ [Myr]}")
ax.tick_params(axis="both", labelsize=fontsize)

# set axis limits
ax.set_xlim(tlim...)
ax.set_ylim(dtlim...)
ax.set_yscale("log")

for ax in axs[1:2]
    ax.set_xticklabels([])
end

for ax in axs
    ax.grid(which="major", axis="both", zorder=-3)
    ax.axvline(-14.39, linestyle=":", linewidth=2, color="orange", zorder=0)
    ax.fill_between([-14.39-0.78, -14.39+0.74], [1e4, 1e4], color="orange", zorder=-1, alpha=0.2)

    ax.fill_between([-5.5, -3.5], [1e4, 1e4], color="royalblue", zorder=-1, alpha=0.2)
end

savefig(output_directory * "Histories_$(family).pdf", bbox_inches="tight", dpi=300)
fig.clear()

## C.3 Trajectories in Alpha Per

In [ ]:
# colormap for gas
cnorm = matplotlib.colors.LogNorm(vmin=2e0, vmax=8e1, clip=true)
cmap  = ColorMap(ColorSchemes.gist_yarg.colors)

# norm and color for Supernova Rate
cmap_SNR = ColorScheme(ColorSchemes.inferno.colors)
cmap_SNR2 = ColorMap(ColorSchemes.inferno.colors)
norm_SNR = matplotlib.colors.Normalize(vmin=2e-2, vmax=0.2, clip=true)

# axis limits
xlim = [-400, 400]
ylim = [-400, 400]

# orbit of LSR
x_center = cicular_orbit.(t_orbit)

# Create figure
fig = plt.figure(figsize=(24, 8), num=3, clear=true)

# create axes
gs = matplotlib.gridspec.GridSpec(1, 4)
subplots_adjust(wspace=0.02, hspace=0.0)

# first axis (past 5 Myr)
ax = subplot(gs[1], title="Past 5 Myr")

# plot gas distribution and local bubble
ax.imshow(permutedims(Σ_gas), origin="lower", extent=[-1250, 1250, -1250, 1250], norm=cnorm, cmap=cmap, zorder=0)
ax.imshow(LB, origin="lower", extent=[-500, 500, -500, 500], zorder=1)

# plot line used for geometry cut
ax.plot([-100, 400], [400, 0], color="purple", linestyle="--", linewidth=2)

# get trajectory in time window
# time indices (depend on choice of t_orbit)
itmax = 51
itmin = 1
its = itmax:-1:itmin

# get trajectory of each cluster
traj = [[get_XYZ_hel(o.getOrbit()[it, :]) - x_center[it] for it in its] for o in orbits["αPer"]]
Rate = [[SN_Rates["αPer"][icl][it, 2] for it in its] for icl in eachindex(traj)]
Prob = [[P_active["αPer"][icl][it] for it in its] for icl in eachindex(traj)]

# plot trajectory of each cluster
for icl in eachindex(traj)
    x = [pt[1] for pt in traj[icl]]
    y = [pt[2] for pt in traj[icl]]
    snr = Rate[icl]
    p   = Prob[icl]
    
    # plot line in linesegments indicating time
    for it in eachindex(x)
        if it < length(x)
            ax.plot(x[it:it+1], y[it:it+1], linewidth=2.5 * it/(itmax - itmin), zorder=2, color=get_color(cmap_SNR, sqrt(snr[it] * snr[it+1]), norm=norm_SNR), 
                    alpha=p[it])
        end
    end
end

# decorate axes
ax.set_xticklabels([])
ax.set_yticklabels([])

# set axis limits
ax.set_xlim(xlim...)
ax.set_ylim(ylim...)
#ax.legend(loc="best", frameon=false, handlelength=1.5, fontsize=fontsize)
ax.tick_params(axis="both", labelsize=fontsize)

# ruler
ax.plot([-350, -150], [-350, -350], color="black", linewidth=2)
ax.annotate("200 pc", xy=(-325, -325), fontsize=fontsize, color="black")

# number of SNe
ax.annotate(L"\mathrm{%$(N_SN_5[2])^{+%$(N_SN_5[3] - N_SN_5[2])}_{-%$(N_SN_5[2] - N_SN_5[1])} \ SNe}", xy=(125, -325), fontsize=fontsize, color="blue")

# next axis (past 5 - 10 Myr)
ax = subplot(gs[2], title="Past 10 - 5 Myr")

# plot gas distribution and local bubble
ax.imshow(permutedims(Σ_gas), origin="lower", extent=[-1250, 1250, -1250, 1250], norm=cnorm, cmap=cmap, zorder=0)
ax.imshow(LB, origin="lower", extent=[-500, 500, -500, 500], zorder=1)

# plot line used for geometry cut
ax.plot([-100, 400], [400, 0], color="purple", linestyle="--", linewidth=2)

# get trajectory in time window
# time indices (depend on choice of t_orbit)
itmax = 101
itmin = 51
its = itmax:-1:itmin

# get trajectory of each cluster
traj = [[get_XYZ_hel(o.getOrbit()[it, :]) - x_center[it] for it in its] for o in orbits["αPer"]]
Rate = [[SN_Rates["αPer"][icl][it, 2] for it in its] for icl in eachindex(traj)]
Prob = [[P_active["αPer"][icl][it] for it in its] for icl in eachindex(traj)]

# plot trajectory of each cluster
for icl in eachindex(traj)
    x = [pt[1] for pt in traj[icl]]
    y = [pt[2] for pt in traj[icl]]
    snr = Rate[icl]
    p   = Prob[icl]
    
    # plot line in linesegments indicating time
    for it in eachindex(x)
        if it < length(x)
            ax.plot(x[it:it+1], y[it:it+1], linewidth=2.5 * it/(itmax - itmin), zorder=2, color=get_color(cmap_SNR, sqrt(snr[it] * snr[it+1]), norm=norm_SNR), 
                    alpha=p[it])
        end
    end
end

# decorate axes
ax.set_xticklabels([])
ax.set_yticklabels([])

# set axis limits
ax.set_xlim(xlim...)
ax.set_ylim(ylim...)
#ax.legend(loc="best", frameon=false, handlelength=1.5, fontsize=fontsize)
ax.tick_params(axis="both", labelsize=fontsize)

# number of SNe
ax.annotate(L"\mathrm{%$(N_SN_10[2])^{+%$(N_SN_10[3] - N_SN_10[2])}_{-%$(N_SN_10[2] - N_SN_10[1])} \ SNe}", xy=(150, -325), fontsize=fontsize, color="blue")

# colorbar
sm   = plt.cm.ScalarMappable(cmap=cmap_SNR2, norm=norm_SNR)
s    = ax.get_position()
cbaxes = fig.add_axes([s.x0 + 0.05 * s.width, s.y0 + 0.15 * s.height, 0.4 * s.width, 0.03 * s.height])
cb = colorbar(sm, cax=cbaxes, orientation="horizontal")
cb.set_label(label=L"\mathrm{SN \ Rate \ [Myr^{-1}]}", size=fontsize, color="black")
cb.ax.tick_params(axis="x", labelsize=0.8fontsize, labelcolor="black")

# next axis (past 10 - 15 Myr)
ax = subplot(gs[3], title="Past 15 - 10 Myr")

# plot gas distribution and local bubble
ax.imshow(permutedims(Σ_gas), origin="lower", extent=[-1250, 1250, -1250, 1250], norm=cnorm, cmap=cmap, zorder=0)
ax.imshow(LB, origin="lower", extent=[-500, 500, -500, 500], zorder=1)

# plot line used for geometry cut
ax.plot([-100, 400], [400, 0], color="purple", linestyle="--", linewidth=2)

# get trajectory in time window
# time indices (depend on choice of t_orbit)
itmax = 151
itmin = 101
its = itmax:-1:itmin

# get trajectory of each cluster
traj = [[get_XYZ_hel(o.getOrbit()[it, :]) - x_center[it] for it in its] for o in orbits["αPer"]]
Rate = [[SN_Rates["αPer"][icl][it, 2] for it in its] for icl in eachindex(traj)]
Prob = [[P_active["αPer"][icl][it] for it in its] for icl in eachindex(traj)]

# plot trajectory of each cluster
for icl in eachindex(traj)
    x = [pt[1] for pt in traj[icl]]
    y = [pt[2] for pt in traj[icl]]
    snr = Rate[icl]
    p   = Prob[icl]
    
    # plot line in linesegments indicating time
    for it in eachindex(x)
        if it < length(x)
            ax.plot(x[it:it+1], y[it:it+1], linewidth=2.5 * it/(itmax - itmin), zorder=2, color=get_color(cmap_SNR, sqrt(snr[it] * snr[it+1]), norm=norm_SNR), 
                    alpha=p[it])
        end
    end
    
end

# decorate axes
ax.set_xticklabels([])
ax.set_yticklabels([])

# set axis limits
ax.set_xlim(xlim...)
ax.set_ylim(ylim...)
#ax.legend(loc="best", frameon=false, handlelength=1.5, fontsize=fontsize)
ax.tick_params(axis="both", labelsize=fontsize)

# number of SNe
ax.annotate(L"\mathrm{%$(N_SN_15[2])^{+%$(N_SN_15[3] - N_SN_15[2])}_{-%$(N_SN_15[2] - N_SN_15[1])} \ SNe}", xy=(150, -325), fontsize=fontsize, color="blue")

# next axis (past 15 - 20 Myr)
ax = subplot(gs[4], title="Past 20 - 15 Myr")

# plot gas distribution and local bubble
ax.imshow(permutedims(Σ_gas), origin="lower", extent=[-1250, 1250, -1250, 1250], norm=cnorm, cmap=cmap, zorder=0)
ax.imshow(LB, origin="lower", extent=[-500, 500, -500, 500], zorder=1)

# plot line used for geometry cut
ax.plot([-100, 400], [400, 0], color="purple", linestyle="--", linewidth=2)

# get trajectory in time window
# time indices (depend on choice of t_orbit)
itmax = 201
itmin = 151
its = itmax:-1:itmin

# get trajectory of each cluster
traj = [[get_XYZ_hel(o.getOrbit()[it, :]) - x_center[it] for it in its] for o in orbits["αPer"]]
Rate = [[SN_Rates["αPer"][icl][it, 2] for it in its] for icl in eachindex(traj)]
Prob = [[P_active["αPer"][icl][it] for it in its] for icl in eachindex(traj)]

# plot trajectory of each cluster
for icl in eachindex(traj)
    x = [pt[1] for pt in traj[icl]]
    y = [pt[2] for pt in traj[icl]]
    snr = Rate[icl]
    p   = Prob[icl]
    
    # plot line in linesegments indicating time
    for it in eachindex(x)
        if it < length(x)
            ax.plot(x[it:it+1], y[it:it+1], linewidth=2.5 * it/(itmax - itmin), zorder=2, color=get_color(cmap_SNR, sqrt(snr[it] * snr[it+1]), norm=norm_SNR), 
                    alpha=p[it])
        end
    end 
end

# decorate axes
ax.set_xticklabels([])
ax.set_yticklabels([])

# set axis limits
ax.set_xlim(xlim...)
ax.set_ylim(ylim...)
#ax.legend(loc="best", frameon=false, handlelength=1.5, fontsize=fontsize)
ax.tick_params(axis="both", labelsize=fontsize)

# number of SNe
ax.annotate(L"\mathrm{%$(N_SN_20[2])^{+%$(N_SN_20[3] - N_SN_20[2])}_{-%$(N_SN_20[2] - N_SN_20[1])} \ SNe}", xy=(150, -325), fontsize=fontsize, color="blue")

savefig(output_directory * "Trajectories_αPer.pdf", bbox_inches="tight", dpi=300)
fig.clear()